In [2]:
import mediapipe as mp
import cv2
import os
import pandas as pd

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def extract_landmarks_from_directory(directory):
    data = []
    labels = []
    for pose_name in os.listdir(directory):
        pose_path = os.path.join(directory, pose_name)
        if os.path.isdir(pose_path):
            for image_file in os.listdir(pose_path):
                image_path = os.path.join(pose_path, image_file)
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Could not read image {image_path}")
                    continue
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = pose.process(image_rgb)
                if results.pose_landmarks:
                    # Extract 33 landmarks (x, y, z for each)
                    landmarks = results.pose_landmarks.landmark
                    data.append([coord for landmark in landmarks for coord in (landmark.x, landmark.y, landmark.z)])
                    labels.append(pose_name)
    return data, labels

# Paths to TRAIN and TEST directories
train_dir = r"C:\Users\hp\Downloads\DATASET\TRAIN"
test_dir = r"C:\Users\hp\Downloads\DATASET\TEST"

# Extract landmarks and labels
train_data, train_labels = extract_landmarks_from_directory(train_dir)
test_data, test_labels = extract_landmarks_from_directory(test_dir)

# Save as DataFrame for later use
train_df = pd.DataFrame(train_data)
train_df['label'] = train_labels

test_df = pd.DataFrame(test_data)
test_df['label'] = test_labels

# Save to CSV for backup
train_df.to_csv('train_landmarks.csv', index=False)
test_df.to_csv('test_landmarks.csv', index=False)

print("Feature extraction completed. Saved to CSV files.")


Feature extraction completed. Saved to CSV files.


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Load extracted data
train_df = pd.read_csv('train_landmarks.csv')
test_df = pd.read_csv('test_landmarks.csv')

# Separate features and labels
X_train = train_df.iloc[:, :-1]  # All columns except the last one
y_train = train_df['label']      # Last column

X_test = test_df.iloc[:, :-1]
y_test = test_df['label']

# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8620689655172413
Classification Report:
               precision    recall  f1-score   support

     downdog       0.97      0.92      0.94        91
     goddess       0.85      0.58      0.69        76
       plank       0.82      0.98      0.89       102
        tree       0.86      0.87      0.86        62
    warrior2       0.84      0.89      0.87       104

    accuracy                           0.86       435
   macro avg       0.87      0.85      0.85       435
weighted avg       0.86      0.86      0.86       435



In [4]:
import joblib

# Save the trained model
joblib.dump(clf, 'yoga_pose_model.pkl')
print("Model saved as 'yoga_pose_model.pkl'.")


Model saved as 'yoga_pose_model.pkl'.


In [ ]:
import warnings

# Ignore sklearn warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")
import joblib
import numpy as np
import cv2
import mediapipe as mp

# Load the trained model
clf = joblib.load('yoga_pose_model.pkl')

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Open webcam feed
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Detect pose
    results = pose.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Extract landmarks
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        data = np.array([[landmark.x, landmark.y, landmark.z] for landmark in landmarks]).flatten()

        # Ensure input is in the same format as the training data (NumPy array)
        input_data = np.array([data])

        # Predict pose
        prediction = clf.predict(input_data)[0]

        # Display predicted pose
        cv2.putText(image, f'Pose: {prediction}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show video feed
    cv2.imshow('Yoga Pose Detection', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
